In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib

from keras.models import Model


In [ ]:

nii_img = nib.load('Task06_Lung/imagesTr/lung_001.nii.gz')
nii_data = nii_img.get_fdata()

nii_labels = nib.load('Task06_Lung/labelsTr/lung_001.nii.gz')
nii_labels_data = nii_labels.get_fdata()


In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    #print(s1.shape, s2.shape, s3.shape, s4.shape)
    #print(p1.shape, p2.shape, p3.shape, p4.shape)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

In [ ]:
for i in range(0, 300):
    plt.subplot(1, 2, 1)
    plt.imshow(nii_data[:, :, i], cmap='gray')
    plt.subplot(1, 2, 2)
    plt.imshow(nii_labels_data[:, :, i+1], cmap='gray')
    plt.show()